In [2]:
# Loading libraries for the course
from skimage import io as ski_io
import random
import matplotlib.pyplot as plt             # Library used for plotting
from matplotlib.patches import Rectangle    # Module to plot a rectangle in the image
import urllib.request                       # Library to download data
import numpy as np                          # Library for array manipulation
import seaborn as sn                        # Library for advanced plotting
import pandas as pd                         # Library to manipulate data frames
import tifffile                             # Library to store numpy arrays as TIFF
import pathlib                              # Library to work with file paths
from pathlib import Path                    # Library to work with file paths
import skimage                              # Library for image manipulation. scikit-image
from skimage.io import imread               # Module from skimage
import skimage as ski
from matplotlib import animation            # Module to plot animations
from skimage.filters import difference_of_gaussians
from skimage import io, measure               # Module from skimage
import numpy as np
import matplotlib.pyplot as plt
from skimage import io as ski_io
from cellpose import models
import pandas as pd
import os



In [3]:
import numpy as np
import pandas as pd
from skimage import io as ski_io, measure
from skimage.filters import difference_of_gaussians
from cellpose import models

# Initialize Cellpose models for cytoplasm and nuclei detection
model_cyto = models.Cellpose(gpu=False, model_type='cyto')
model_nuclei = models.Cellpose(gpu=False, model_type='nuclei')

def DRUG_E(img, model_cyto, model_nuclei):
    # Create an empty DataFrame to store results for all frames
    df_merged_all_frames = pd.DataFrame(columns=[
        'time', 'cell_ID', 'sum_pix_intensity_cyto', 'sum_pix_intensity_nuclei', 
        'cell sizes', 'nucleus sizes', 'Particles in cytosol', 'Particles in nucleus'
    ])

    df_merged_all_frames_transition_sites = pd.DataFrame(columns=[
        'Time', 'Transcription_Site_ID', 'Cell_ID', 'Position_X', 'Position_Y', 'Intensity'
    ])

    transcription_data = []  # List to store transcription site data
    transcription_site_id = 1  # Initialize transcription site ID

    # Iterate over each frame in the image
    for selected_frame in range(len(img)):
        # Select the appropriate frame and color channel for cytoplasm detection
        img_to_analyze = img[selected_frame, :, :, 1]

        # Run Cellpose for cytoplasm detection
        masks_cyto, _, _, _ = model_cyto.eval(img_to_analyze, diameter=None, channels=[0, 0])

        if selected_frame == 0:
            # Save the initial cytoplasm mask for reference
            mask_initial_cyto = masks_cyto

        # Adjust masks for cytoplasm based on initial mask
        masks_cyto = np.where(masks_cyto != 0, (masks_cyto * mask_initial_cyto) / masks_cyto, 0)
        
        # Prepare data for the cytoplasm DataFrame
        data_cyto = []
        for cell_id in np.unique(masks_cyto):
            if cell_id == 0:
                continue  # Skip background
            mask = masks_cyto == cell_id
            sum_pix_intensity = img_to_analyze[mask].sum()
            cell_sizes = np.sum(mask)

            data_cyto.append([selected_frame, cell_id, sum_pix_intensity, cell_sizes])

        df_cyto = pd.DataFrame(data_cyto, columns=['time', 'cell_ID', 'sum_pix_intensity_cyto', 'cell sizes'])

        # Select the appropriate frame and color channel for nuclei detection
        img_to_analyze = img[selected_frame, :, :, 0]

        # Run Cellpose for nuclei detection
        masks_nuclei, _, _, _ = model_nuclei.eval(img_to_analyze, diameter=None, channels=[0, 0])

        if selected_frame == 0:
            # Save the initial nuclei mask for reference
            mask_initial_nuclei = masks_nuclei

        # Adjust masks for nuclei based on initial mask
        masks_nuclei = np.where(masks_nuclei != 0, (masks_nuclei * mask_initial_nuclei) / masks_nuclei, 0)

        # Prepare data for the nuclei DataFrame
        data_nuclei = []
        for cell_id in np.unique(masks_nuclei):
            if cell_id == 0:
                continue  # Skip background
            mask = masks_nuclei == cell_id
            nucleus_sizes = np.sum(mask)
            sum_pix_intensity = img_to_analyze[mask].sum()

            data_nuclei.append([selected_frame, cell_id, nucleus_sizes, sum_pix_intensity])

        df_nuclei = pd.DataFrame(data_nuclei, columns=['time', 'cell_ID', 'nucleus sizes', 'sum_pix_intensity_nuclei'])

        # Merge the Cytoplasm and Nuclei DataFrames based on the 'cell_ID' column
        df_merged = pd.merge(df_cyto, df_nuclei, on='cell_ID')
        df_merged['time'] = df_merged['time_x']
        df_merged = df_merged.drop(columns=['time_x', 'time_y'])
        df_merged = df_merged[['time', 'cell_ID', 'sum_pix_intensity_cyto', 'sum_pix_intensity_nuclei', 'cell sizes', 'nucleus sizes']]
        
        # Calculate protein concentrations (AU)
        df_merged['protein concentrations (AU)'] = (
            df_merged['sum_pix_intensity_cyto'] - df_merged['sum_pix_intensity_nuclei']
        ) / (df_merged['cell sizes'] - df_merged['nucleus sizes'])

        # Concatenate DataFrames for all frames
        df_merged_all_frames = pd.concat([df_merged_all_frames, df_merged], ignore_index=True)

        # Analyze RNA spots
        selected_color_channel = 2
        img_to_analyze = img[selected_frame, :, :, selected_color_channel]

        # Apply a difference of Gaussians filter to enhance spots
        img_spots_filtered = difference_of_gaussians(img_to_analyze, low_sigma=1, high_sigma=5)
        
        # Create a binary image by applying a threshold
        threshold = 0.05
        img_spots_binary = np.where(img_spots_filtered >= threshold, 1, 0)

        # Find contours of spots in the binary image
        spot_contours = measure.find_contours(img_spots_binary, 0.5)

        number_detected_cells = np.max(mask_initial_cyto)
        list_cytosol_particles = np.zeros(number_detected_cells)
        list_nuclear_particles = np.zeros(number_detected_cells)

        # Loop through the spot contours
        for contour in spot_contours:
            # Find the position of the particle
            posn = np.mean(contour, axis=0).astype(int)

            # Ensure the position is within the image bounds
            if posn[0] >= img_spots_binary.shape[0] or posn[1] >= img_spots_binary.shape[1]:
                continue
            
            # Check which cytoplasm mask the particle is in
            cell_num = mask_initial_cyto[posn[0], posn[1]]
            if cell_num > 0:
                list_cytosol_particles[cell_num - 1] += 1

                # Check if the particle is also in the nucleus
                if mask_initial_nuclei[posn[0], posn[1]] == cell_num:
                    list_nuclear_particles[cell_num - 1] += 1

                
        # Add the number of particles to the merged DataFrame
        df_merged['Particles in cytosol'] = list_cytosol_particles[df_merged['cell_ID'].astype(int) - 1]
        df_merged['Particles in nucleus'] = list_nuclear_particles[df_merged['cell_ID'].astype(int) - 1]

        # Concatenate DataFrames
        df_merged_all_frames = pd.concat([df_merged_all_frames, df_merged], ignore_index=True)
        df_merged_all_frames.dropna(inplace=True)

        selected_color_channel = 2
        img_to_analyze = img[selected_frame, :, :, selected_color_channel]

        # Apply a difference of Gaussians filter to the image to enhance spots
        img_spots_filtered = difference_of_gaussians(img_to_analyze, low_sigma=1, high_sigma=5)

        # Apply a threshold to the image to create a binary image
        threshold = 0.09
        img_spots_binary = np.where(img_spots_filtered >= threshold, 1, 0)

        # Find contours in the binary image
        spot_contours = measure.find_contours(img_spots_binary, 0.5)

        # Initialize lists to keep track of transcription sites and their details
        number_detected_cells = np.max(mask_initial_cyto)
        list_transcription_sites = np.zeros(number_detected_cells, dtype=int)
        list_position_transcription_sites = [[] for _ in range(number_detected_cells)]
        list_intensity_transcription_sites = [[] for _ in range(number_detected_cells)]

        # Loop through the spot contours
        for contour in spot_contours:
            # Find the position of the particle
            posn = np.mean(contour, axis=0).astype(int)
            
            # Ensure the position is within the image bounds
            if posn[0] >= img_spots_binary.shape[0] or posn[1] >= img_spots_binary.shape[1]:
                continue
                
            # Check which nuclei mask the particle is in
            cell_num = mask_initial_nuclei[posn[0], posn[1]]
            if cell_num > 0:
                list_transcription_sites[cell_num - 1] += 1
                list_position_transcription_sites[cell_num - 1].append(posn)
                intensity_transcription_sites = img_to_analyze[posn[0], posn[1]]
                list_intensity_transcription_sites[cell_num - 1].append(intensity_transcription_sites)

        # Create a list of dictionaries to construct the DataFrame
        transcription_data = []
        transcription_site_id = 1
        for cell_id in range(number_detected_cells):
            for posn, intensity in zip(list_position_transcription_sites[cell_id], list_intensity_transcription_sites[cell_id]):
                transcription_data.append({
                    'Time' : selected_frame,
                    'Transcription_Site_ID': transcription_site_id,
                    'Cell_ID': cell_id + 1,
                    'Position_X': posn[0],
                    'Position_Y': posn[1],
                    'Intensity': intensity
                })
                transcription_site_id += 1
        

        # Create a DataFrame for transcription sites
        df_transcription_sites_all = pd.DataFrame(transcription_data)

        # Add df_transcription_sites_all to the bigger dataframe, with all frames
        df_merged_all_frames_transition_sites = pd.concat([df_merged_all_frames_transition_sites, df_transcription_sites_all], ignore_index=True)

    return df_merged_all_frames, df_merged_all_frames_transition_sites


Code for Daria to analysis

In [4]:
# Load all images and process them
image_directory = r'C:\Users\daria\OneDrive - Politechnika Śląska\QBio\CourseMaterials\uqbio2024\UQbio2024_Team6\Drug2_1000nM'
image_files = os.listdir(image_directory)
df_Drug2_1000nM_cell = pd.DataFrame()
df_Drug2_1000nM_transcription_sites = pd.DataFrame()

for image_file in image_files:
    if image_file.endswith('.tif'):
        image_path = os.path.join(image_directory, image_file)
        img = ski_io.imread(image_path)
        df_merged_all_frames, df_transcription_sites = DRUG_E(img, model_cyto, model_nuclei)
        df_merged_all_frames['image_name'] = image_file
        df_transcription_sites['image_name'] = image_file
        df_Drug2_1000nM_cell = pd.concat([df_Drug2_1000nM_cell, df_merged_all_frames], ignore_index=True)
        df_Drug2_1000nM_transcription_sites = pd.concat([df_Drug2_1000nM_transcription_sites, df_transcription_sites], ignore_index=True)

# Save the merged DataFrame to a CSV file
csv_file_path = 'df_Drug2_1000nM_cell_new.csv'
df_Drug2_1000nM_cell.to_csv(csv_file_path, index=False)

csv_file_path = 'df_Drug2_1000nM_transcription_sites.csv'
df_Drug2_1000nM_transcription_sites.to_csv(csv_file_path, index=False)


C:\Users\daria\AppData\Local\Temp\ipykernel_16780\1169115017.py:38: RuntimeWarning: invalid value encountered in divide
  masks_cyto = np.where(masks_cyto != 0, (masks_cyto * mask_initial_cyto) / masks_cyto, 0)
C:\Users\daria\AppData\Local\Temp\ipykernel_16780\1169115017.py:64: RuntimeWarning: invalid value encountered in divide
  masks_nuclei = np.where(masks_nuclei != 0, (masks_nuclei * mask_initial_nuclei) / masks_nuclei, 0)
C:\Users\daria\AppData\Local\Temp\ipykernel_16780\1169115017.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_merged_all_frames = pd.concat([df_merged_all_frames, df_merged], ignore_index=True)
C:\Users\daria\AppData\Local\Temp\ipykernel_16780\1169115017.py:135: FutureWarning: The behavior of DataFrame concate

Code for Kristi

In [5]:
# Load all images and process them
image_directory = r'C:\Users\daria\OneDrive - Politechnika Śląska\QBio\CourseMaterials\uqbio2024\UQbio2024_Team6\Drug6_1000nM'
image_files = os.listdir(image_directory)
df_Drug6_1000nM_cell = pd.DataFrame()
df_Drug6_1000nM_transcription_sites = pd.DataFrame()

for image_file in image_files:
    if image_file.endswith('.tif'):
        image_path = os.path.join(image_directory, image_file)
        img = ski_io.imread(image_path)
        df_merged_all_frames, df_transcription_sites = DRUG_E(img, model_cyto, model_nuclei)
        df_merged_all_frames['image_name'] = image_file
        df_transcription_sites['image_name'] = image_file
        df_Drug6_1000nM_cell = pd.concat([df_Drug6_1000nM_cell, df_merged_all_frames], ignore_index=True)
        df_Drug6_1000nM_transcription_sites = pd.concat([df_Drug6_1000nM_transcription_sites, df_transcription_sites], ignore_index=True)

# Save the merged DataFrame to a CSV file
csv_file_path = 'df_Drug6_1000nM_cell.csv'
df_Drug6_1000nM_cell.to_csv(csv_file_path, index=False)

csv_file_path = 'df_Drug6_1000nM_transcription_sites.csv'
df_Drug6_1000nM_transcription_sites.to_csv(csv_file_path, index=False)



C:\Users\daria\AppData\Local\Temp\ipykernel_16780\1169115017.py:38: RuntimeWarning: invalid value encountered in divide
  masks_cyto = np.where(masks_cyto != 0, (masks_cyto * mask_initial_cyto) / masks_cyto, 0)
C:\Users\daria\AppData\Local\Temp\ipykernel_16780\1169115017.py:64: RuntimeWarning: invalid value encountered in divide
  masks_nuclei = np.where(masks_nuclei != 0, (masks_nuclei * mask_initial_nuclei) / masks_nuclei, 0)
C:\Users\daria\AppData\Local\Temp\ipykernel_16780\1169115017.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_merged_all_frames = pd.concat([df_merged_all_frames, df_merged], ignore_index=True)
C:\Users\daria\AppData\Local\Temp\ipykernel_16780\1169115017.py:135: FutureWarning: The behavior of DataFrame concate

Code for Eli

In [6]:
# Load all images and process them
image_directory = r'C:\Users\daria\OneDrive - Politechnika Śląska\QBio\CourseMaterials\uqbio2024\UQbio2024_Team6\Drug7_1000nM'
image_files = os.listdir(image_directory)
df_Drug7_1000nM_cell = pd.DataFrame()
df_Drug7_1000nM_transcription_sites = pd.DataFrame()

for image_file in image_files:
    if image_file.endswith('.tif'):
        image_path = os.path.join(image_directory, image_file)
        img = ski_io.imread(image_path)
        df_merged_all_frames, df_transcription_sites = DRUG_E(img, model_cyto, model_nuclei)
        df_merged_all_frames['image_name'] = image_file
        df_transcription_sites['image_name'] = image_file
        df_Drug7_1000nM_cell = pd.concat([df_Drug7_1000nM_cell, df_merged_all_frames], ignore_index=True)
        df_Drug7_1000nM_transcription_sites = pd.concat([df_Drug7_1000nM_transcription_sites, df_transcription_sites], ignore_index=True)

# Save the merged DataFrame to a CSV file
csv_file_path = 'df_Drug7_1000nM_cell.csv'
df_Drug7_1000nM_cell.to_csv(csv_file_path, index=False)

csv_file_path = 'df_Drug7_1000nM_transcription_sites.csv'
df_Drug7_1000nM_transcription_sites.to_csv(csv_file_path, index=False)

C:\Users\daria\AppData\Local\Temp\ipykernel_16780\1169115017.py:38: RuntimeWarning: invalid value encountered in divide
  masks_cyto = np.where(masks_cyto != 0, (masks_cyto * mask_initial_cyto) / masks_cyto, 0)
C:\Users\daria\AppData\Local\Temp\ipykernel_16780\1169115017.py:64: RuntimeWarning: invalid value encountered in divide
  masks_nuclei = np.where(masks_nuclei != 0, (masks_nuclei * mask_initial_nuclei) / masks_nuclei, 0)
C:\Users\daria\AppData\Local\Temp\ipykernel_16780\1169115017.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_merged_all_frames = pd.concat([df_merged_all_frames, df_merged], ignore_index=True)
C:\Users\daria\AppData\Local\Temp\ipykernel_16780\1169115017.py:135: FutureWarning: The behavior of DataFrame concate